***
____
![revit](https://i.ibb.co/bQ3dB8C/curso-revit.png)

***
***


# Clase 07
## Tunning de modelos

In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10, 10]
np.random.seed(42)

# Optimización de hiperparámetros

Hasta ahora hemos visto una manera relativamente sencilla de ver que valores de los hiperparámetros funcionan mejor, mediante las curvas de validación.

Estas curvas son muy útiles para darnos información a los Data Scientists, pero tienen dos problemas:
- Son métodos gráficos, esto significa que necesitan un humano para interpretarlas y no nos permiten automatizar el proceso para encontrar los hiperparámetros óptimos.
- Solo toman un hiperparámetro a la vez. Esto significa que hacen que sea más dificil el evaluar combinaciones de los hiperparámetros (si quisieramos evaluar multiples hiperparámetros tendriamos que hacer gráficas de planos o hiperplanos).

Vamos a ver ahora métodos más robustos para dado un modelo, encontrar el conjunto de hiperparámetros que hace que funcione mejor.

# Cargamos los datos

Vamos a usar un dataset nuevo, el [Census Income Dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income). Es un dataset que tiene datos demográficos sobre 50,000 personas en Estados Unidos y como variable objetivo tiene una variable booleana (Verdadero/Falso) sobre si dicha persona gana más de 50K$ al año o no.

In [2]:
censo = pd.read_csv("data/salario_censo.csv")

In [3]:
censo.shape

(32561, 13)

In [4]:
censo.head()

,edad,clase_laboral,nivel_educativo,status_matrimonial,ocupacion,relacion,raza,genero,ganancias_capital,perdidas_capital,horas_laborables,pais_origen,objetivo
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Procesamiento de datos

In [5]:
variable_dependiente = "objetivo"

In [6]:
X_inicial = censo.drop(variable_dependiente, axis =1)
y = censo[variable_dependiente]

In [7]:
y.unique()

array([' <=50K', ' >50K'], dtype=object)

En este caso la variable objetivo está definida como texto, asi que la convertimos a una variable binaria numérica.

In [8]:
y = y.replace({" <=50K":0, " >50K":1})

Separamos datos en numéricos y no numéricos. Viendo el [diccionario de datos](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names) del dataset vemos que no hay variables categóricas, solo la variable educacion que ya viene codificada como numérica (*education-num*).

In [9]:
# Separamos los datos numéricos y categóricos
datos_numericos = X_inicial.select_dtypes(include=['float64', "int64"])
datos_categoricos = X_inicial.select_dtypes(exclude=['float64', "int64"])

# Para los missing numéricos los imputamos con la media
for col in datos_numericos.columns:
    datos_numericos[col].fillna(datos_numericos[col].mean(), inplace=True)

# Para los categoricos creamos dummies
datos_categoricos_codificados = pd.get_dummies(datos_categoricos)
X = pd.concat([datos_numericos, datos_categoricos_codificados], axis=1)

C:\Users\Alfy\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
X.shape

(32561, 91)

Antes que nada vamos a ver que puntuaciones tienen unos cuantos modelos con sus  hiperparámetro por defecto

In [11]:
resultados = {}

In [12]:
from sklearn.model_selection import cross_validate

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                     scoring="roc_auc", n_jobs=-1, cv=5, return_train_score=True)
    return resultados_estimador

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].max()
    return resultados_df

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [14]:
resultados["reg_logistica"] = evaluar_modelo(LogisticRegression(), X, y)
resultados["naive_bayes"] = evaluar_modelo(GaussianNB(), X, y)
resultados["rf"] = evaluar_modelo(RandomForestClassifier(), X, y)
resultados["svc"] = evaluar_modelo(SVC(), X, y)

In [15]:
ver_resultados()

,fit_time,score_time,test_score,train_score,fit_time_idx,score_time_idx,test_score_idx,train_score_idx
reg_logistica,0.582841,0.013370,0.905177,0.906081,0.005800,0.000998,1.000000,0.910218
naive_bayes,0.141620,0.037100,0.888876,0.889217,0.001409,0.002768,0.981992,0.893276
rf,0.627322,0.035510,0.874810,0.995456,0.006243,0.002650,0.966452,1.000000
svc,100.482281,13.401541,0.888096,0.910491,1.000000,1.000000,0.981130,0.914647


Vamos a seleccionar un estimador en función de los resultados iniciales y optimizarlo. Elijo el estimador Random Forest por que funciona muy bien en comparación a los demás y es bastánte rápido de entrenar.

In [16]:
estimador_rf = RandomForestClassifier()

Scikit-learn tiene dos métodos de optimización de hiperparámetros, [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) y [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV).

`GridSearchCV` funciona realizando una busqueda en una malla, es decir, pasandole un conjunto de posibles opciones de hiperparámetros evalua de forma completa cada combinación de dichos parámetros (es decir, el valor 1 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, el valor 2 del hiperparámetro 1 combinado con todos los posibles valores de los demás hiperparámetros, etcétera).

La ventaja de utilizar una búsqueda de malla es que nos aseguramos de que se han probado todas las combinaciones posibles. El problema es que el proceso requiere mucho tiempo de computación, y según que dataset usemos 

In [17]:
%%timeit
import time
def foo():
    time.sleep(1)

348 ns ± 5.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%%timeit -n 1 -r 1  #n 1 dice que ejecute esta celda solo una vez, -r 1 que ejecute un solo loop
def foo():
    time.sleep(1)

395 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [20]:
print(estimador_rf.__doc__)

A random forest classifier.

    A random forest is a meta estimator that fits a number of decision tree
    classifiers on various sub-samples of the dataset and uses averaging to
    improve the predictive accuracy and control over-fitting.
    The sub-sample size is always the same as the original
    input sample size but the samples are drawn with replacement if
    `bootstrap=True` (default).

    Read more in the :ref:`User Guide <forest>`.

    Parameters
    ----------
    n_estimators : integer, optional (default=10)
        The number of trees in the forest.

        .. versionchanged:: 0.20
           The default value of ``n_estimators`` will change from 10 in
           version 0.20 to 100 in version 0.22.

    criterion : string, optional (default="gini")
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "entropy" for the information gain.
        Note: this parameter is tree-specific.

    max_depth :

In [21]:
estimador_rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

Vamos a definir los límites de la búsqueda de hiperparámetros.

In [22]:
np.linspace(10,1000,10).astype(int)

array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])

In [1]:
parametros_busqueda_rf = {
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
    "class_weight": [None, "balanced"]
}

NameError: name 'np' is not defined

In [24]:
grid = GridSearchCV(estimator=estimador_rf, 
                    param_grid=parametros_busqueda_rf,
                    scoring="roc_auc", n_jobs=-1)

`GridSearchCV` se comporta como un estimador en cuanto a que tiene un metodo fit que usamos para "entrenarlo" y que realize la búsqueda en malla.

Para ver cuanto tiempo tarda en realizar la búsqueda usamos la mágia de Jupyter notebook `%%timeit` que evalua el tiempo que tarda una función en ejecutarse


In [25]:
%%timeit -n 1 -r 1
grid.fit(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


27min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi ordenador la busqueda en malla ha tardado 7minutos y 49 segundos 

Ahora podemos ver la puntuación que ha obtenido el mejor estimador así como los parámetros del mismo

In [26]:
print(grid.best_score_)
print(grid.best_estimator_)

0.8973525008870069
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=890, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Tras haberlo ajustado, Gridsearch nos devuelve el ranking de todas las variantes evaluadas junto con métricas de su funcionamiento con el atributo `cv_results_`

In [27]:
pd.DataFrame(grid.cv_results_).sort_values(by="rank_test_score")

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\utils\de

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
18,73.756554,0.762596,6.522890,0.166150,None,entropy,890,"{'class_weight': None, 'criterion': 'entropy',...",0.894167,0.896621,0.901270,0.897353,0.002945,1,0.998224,0.998142,0.998036,0.998134,0.000077
19,83.158527,1.006302,7.345021,0.233350,None,entropy,1000,"{'class_weight': None, 'criterion': 'entropy',...",0.894558,0.896468,0.901017,0.897347,0.002710,2,0.998220,0.998156,0.998026,0.998134,0.000080
15,46.816092,0.754700,4.174504,0.046624,None,entropy,560,"{'class_weight': None, 'criterion': 'entropy',...",0.894559,0.896735,0.900441,0.897245,0.002428,3,0.998218,0.998135,0.998015,0.998123,0.000084
16,57.922133,0.914542,5.393932,0.608828,None,entropy,670,"{'class_weight': None, 'criterion': 'entropy',...",0.894220,0.896379,0.900565,0.897055,0.002634,4,0.998218,0.998138,0.998057,0.998137,0.000066
17,69.493612,1.245437,5.739948,0.178525,None,entropy,780,"{'class_weight': None, 'criterion': 'entropy',...",0.894438,0.895837,0.900510,0.896928,0.002596,5,0.998220,0.998149,0.998049,0.998139,0.000070
14,38.281744,0.498377,3.130282,0.075287,None,entropy,450,"{'class_weight': None, 'criterion': 'entropy',...",0.894409,0.896068,0.900281,0.896919,0.002471,6,0.998217,0.998123,0.998014,0.998118,0.000083
8,64.935093,2.875777,6.428460,0.559175,None,gini,890,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893962,0.896232,0.900200,0.896798,0.002578,7,0.998233,0.998146,0.998057,0.998145,0.000072
37,62.566500,0.717926,5.724939,0.092034,balanced,entropy,780,"{'class_weight': 'balanced', 'criterion': 'ent...",0.894605,0.895841,0.899921,0.896789,0.002271,8,0.997586,0.997522,0.997051,0.997386,0.000239
7,53.999356,0.201989,5.053727,0.026956,None,gini,780,"{'class_weight': None, 'criterion': 'gini', 'n...",0.893907,0.896036,0.900377,0.896773,0.002693,9,0.998235,0.998160,0.998052,0.998149,0.000075
9,79.667451,0.231628,7.569657,0.101920,None,gini,1000,"{'class_weight': None, 'criterion': 'gini', 'n...",0.894186,0.895625,0.900408,0.896740,0.002660,10,0.998222,0.998149,0.998031,0.998134,0.000079


`GridSearchCV` al estar ajustado se convierte en un estimador, por lo cual podemos usar el método predict, por debajo simplemente se usará el mejor estimador `grid.best_estimator_`. 

Para añadir el funcionamiento del mejor estimador obtenido por el modelo con nuestra funcion `evaluar_modelo` no usamos el objeto grid en si, ya que la funcion `cross_validate` hace multiples ajustes y evaluaciones (volveriamos a esperar los 8 minutos que a tardado un ajuste multiplicado por el número de validaciones cruzadas!).

Para evaluar el funcionamiento del mejor estimador simplemente usamos la funcion con el mejor estimador directamente.

In [28]:
resultados["rf_gridsearch"] = evaluar_modelo(grid.best_estimator_, X, y)

Ahora vamos a realizar la misma optimización de parámetros pero usando `RandomizedSearchCV`. RandomizedSearchCV funciona de forma similar a GridSearchCV, pero en vez de evaluar todas las combinaciones posibles de hiperparámetros, se toman n muestras de hiperparámetros de dichas distribuciones.

Se recomienda usar distribuciones en vez de valores fijos para hiperparámetros continuos.

Primero vamos a evaluar el funcionamiento de la busqueda aleatoria con los mísmos hiperparámetros que hemos usado en la busqueda en malla. Para `RandomizedSearchCV` tenemos que indicarle cuantas variantes de hiperparámetros utilizar (definidas por el parámetro n_iter, por defecto toma 10 variantes). Dado que dicha búsqueda toma muestreos el parámetro ya no se llama `param_grid` sino `param_distributions`.

In [29]:
busqueda_random = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=parametros_busqueda_rf,
                   scoring="roc_auc", n_jobs=-1, n_iter=10)

In [30]:
%%timeit -n 1 -r 1
busqueda_random.fit(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


15min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


La búsqueda con 10 iteraciones ha tardado 1min 25s en mi máquina. Veamos como ha funcionado.

In [31]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.8968929619020128
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=560, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


La búsqueda de malla obtuvo un ROC AUC máximo de 0.89726431782 versus 0.896926092788 obtenido por la búsqueda aleatoria. Sin embargo la busqueda aleatoria ha tardado 8 veces menos!

In [32]:
resultados["rf_randomizedsearch"] = evaluar_modelo(grid.best_estimator_, X, y)

Una ventaja del Randomized Search es que nos permite evaluar un espacio de hiperparámetros más amplio para un tiempo de computación similar.

Para ver esto vamos a ampliar el espacio de búsqueda de hiperparámetros y hacer 100 muestreos.

In [33]:
from scipy.stats import randint as sp_randint

param_dist_random = {
    "max_depth": [3, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
}

In [34]:
busqueda_random_100 = RandomizedSearchCV(estimator=estimador_rf, 
                    param_distributions=param_dist_random,
                   scoring="roc_auc", n_jobs=-1, n_iter=100)

In [35]:
%%timeit -n 1 -r 1
busqueda_random_100.fit(X, y)

C:\Users\Alfy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


13min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


En mi máquina esta búsqueda ha tardado 8 minutos 54 segundos, un poco más que el grid search

In [36]:
print(busqueda_random_100.best_score_)
print(busqueda_random_100.best_estimator_)

0.9193261285855976
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


La búsqueda aleatoria con los nuevos parámetros ha tardado un tiempo similar a la busqueda en malla, pero ha obtenido una puntuación máxima ROC AUC de 0.91950285418 (versus 0.89726431782 de la busqueda en malla)

In [37]:
resultados["rf_randomizedsearch_100"] = evaluar_modelo(busqueda_random_100.best_estimator_,
                                                      X, y)

Vemos que el estimador obtenido con la búsqueda aleatoria es el que mejor funciona.

En general, salvo que el espacio de hiperparámetros que queramos explorar sea pequeño, es mejor el utilizar `RandomizedSearchCV` en vez de `GridSearchCV`. Esto es así por que en general no existe un unico conjunto de hiperparámetros que obtiene el mejor funcionamiento, sino que suelen existir multiples "areas" en el espacio dimensional de los hiperparámetros que funcionan de forma similar. Al hacer una búsqueda aleatoria podemos explorar las diversas areas en un tiempo más reducido.